In [26]:
import matplotlib.pyplot as plt
import numpy as np

In [27]:
np.set_printoptions(precision=8)

In [28]:
path="/Users/ecasiano/Desktop/PrototypeScripts/TestSystematicError/"
path="/Users/ecasiano/XCode/pimc/pimc/Build/pimc/"

In [29]:
seed = 326

In [30]:
data_SWAPn_m2 = np.loadtxt(path+"1D_4_4_2_3.300000_1.000000_5.000000_1000_SWAPn-mA2_"+str(seed)+"_square.dat")
data_PnSquared_m2 = np.loadtxt(path+"1D_4_4_2_3.300000_1.000000_5.000000_1000_PnSquared-mA2_"+str(seed)+"_square.dat")

data_SWAPn_m2.shape,data_PnSquared_m2.shape

((4500, 5), (4500, 5))

In [31]:
N=data_SWAPn_m2.shape[1]-1 # Get N

In [32]:
S2 = np.zeros(data_SWAPn_m2.shape)
SWAPn_m2_col_sum = np.sum(data_SWAPn_m2,axis=0)
PnSquared_m2_col_sum = np.sum(data_PnSquared_m2,axis=0)
for i in range(data_SWAPn_m2.shape[0]):
    PnSquared_m2_col_sum_i = PnSquared_m2_col_sum - data_PnSquared_m2[i]
    
    SWAPn_m2_col_sum_i = SWAPn_m2_col_sum - data_SWAPn_m2[i]
    
    S2[i] = -np.log(SWAPn_m2_col_sum_i/PnSquared_m2_col_sum_i)

In [33]:
S2_mean = np.mean(S2,axis=0)
S2_mean

array([-0.04662885, -0.00800725,  0.01125783,  0.02744293, -1.22772419])

In [34]:
N_rows = S2.shape[0]

In [35]:
S2_err = np.std(S2,axis=0)
S2_err *= np.sqrt(N_rows-1)

In [36]:
for n in range(N+1):
    print(f"<S2(n={n})> = {S2_mean[n]:0.6f} +/- {S2_err[n]:0.6f}")

<S2(n=0)> = -0.046629 +/- 0.039363
<S2(n=1)> = -0.008007 +/- 0.025387
<S2(n=2)> = 0.011258 +/- 0.007921
<S2(n=3)> = 0.027443 +/- 0.027234
<S2(n=4)> = -1.227724 +/- 0.781340


In [37]:
data_Pn_m2 = np.loadtxt(path+"1D_4_4_2_3.300000_1.000000_5.000000_1000_Pn-mA2_"+str(seed)+"_square.dat")
data_Pn_m2.shape

(4500, 5)

In [38]:
data_Pn_col_sum_m2 = np.sum(data_Pn_m2,axis=0)

In [39]:
jacknifed_Pn = np.zeros(data_Pn_m2.shape)
for i in range(data_Pn_m2.shape[0]):
    data_Pn_col_sum_m2_i = data_Pn_col_sum_m2 - data_Pn_m2[i]
        
    jacknifed_Pn[i] = data_Pn_col_sum_m2_i
jacknifed_Pn /= np.sum(jacknifed_Pn,axis=1)[:,None]

In [40]:
Pn_mean = np.mean(jacknifed_Pn,axis=0)
Pn_mean /= np.sum(Pn_mean )
N_rows_Pn = jacknifed_Pn.shape[0]
Pn_err = np.std(jacknifed_Pn,axis=0)
Pn_err *= np.sqrt(N_rows_Pn-1)

In [41]:
for n in range(N+1):
    print(f'P(n={n}) = {Pn_mean[n]:0.4f} +/- {Pn_err[n]:0.4f}')

P(n=0) = 0.0100 +/- 0.0004
P(n=1) = 0.1966 +/- 0.0014
P(n=2) = 0.5899 +/- 0.0017
P(n=3) = 0.1934 +/- 0.0014
P(n=4) = 0.0100 +/- 0.0004


# 